## Note : 
Mean free path : $\lambda = \frac{1}{n \sigma}$

If $n \rightarrow 100 \times n$, then :
- $\lambda \rightarrow \frac{\lambda}{100}$
- $\text{cell size} \rightarrow \frac{\text{cell size}}{100}$ ($100^2$ more cells because the grid is 2D)
- $dt \rightarrow \frac{dt}{100}$

But you can lower, down to a certain extent, the number of particles per cell to "not incease too much the computation time".

## Representative physical length scale

The system has two lenghts : 
* lenght of the tube : $L = 0.01$ $m$
* width : $w = 0.001$ $m$

The width is the representative physical length scale that constrains the flow.

So : $K_d = \frac{\lambda}{w}$.




# ISP, thrust etc.

In [1]:
def isp(ve):
    return ve/9.81

def thrust(mass, speeds, dt):
    return mass*np.sum(speeds)/dt

In [2]:

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
%matplotlib widget

# notebook
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy as scipy
from pathlib import Path
import seaborn as sns

from lppydsmc.data.saver import Saver
from plotting import analysis
from lppydsmc.utils import physics

# plt.figure.max_open_warning = False

## Loading results

In [4]:
dir_path = Path('results/')
dz = 0.001

In [5]:
dt = 1e-8
mr = 1
name = 'neutralization_3_minus100.h5'
lenght = 0.011 # m
width = 0.005
cell_size = 0.001
temperature = 23e3 # K
nb_cells = 41 # 33
volume = cell_size*cell_size*dz*nb_cells

nb_injected_particles = 10000

In [6]:
store = pd.HDFStore(dir_path/name)

In [7]:
store.keys()

['/collisions_with_walls',
 '/df',
 '/df_collision_background_gas',
 '/df_collision_with_walls',
 '/df_out_particles',
 '/mean_proba_gas',
 '/mean_proba_walls',
 '/total_deleted']

In [8]:
# defining each variable
df = store['df']
df_out_particles = store['df_out_particles']
df_collision_background_gas = store['df_collision_background_gas']
df_collision_with_walls = store['df_collision_with_walls']
total_deleted = store['total_deleted']
collisions_with_walls = store['collisions_with_walls']
mean_proba_walls = store['mean_proba_walls']
mean_proba_gas = store['mean_proba_gas']

In [9]:
choice = 'I'
gamma = 5/3. # roughly for atomes, for diatomic molecule : 7/5.


mass = physics.get_mass_part(53+1, 53, 74) # I
molecular_mass = 0.1269 # kg/mol

verbose=False

In [10]:
sound_vel = physics.speed_of_sound(molecular_mass, temperature, gamma)
print(f'Speed of sound for [{choice}] at temperature {temperature} K : {sound_vel} m/s')

Speed of sound for [I] at temperature 23000.0 K : 1584.7563179678798 m/s


## Note : 
To save a figure, you can use : `analysis.save_fig(fig, path, title = None, dpi = 400, figsize = None)`, which allows you to easily save a figure while giving it a title and changing its size.
You can also use the interactive widget.

## Choosing the part of the system to plot 

In [11]:

res_x_0, res_y_0 = 11, 5

total_nb_cells = res_x_0*res_y_0

In [12]:
fig, ax = plt.subplots(1, constrained_layout = True, figsize = (res_x_0, res_y_0));

fact = 10
volume_cell_plot = volume/(fact*fact*res_x_0*res_y_0)
duration = df.index.unique().values.shape[0]
analysis.hist2d(ax, df, bins = (fact*res_x_0,fact*res_y_0), weights = mr*np.ones(df.shape[0])/(duration*volume_cell_plot), stat = 'count'); # TODO: add cmap

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Choosing frames to plot

In [13]:
unique_index = df.index.unique().values
nb_save = unique_index.shape[0]
iterations = np.max(unique_index)
adding_period = unique_index[1]-unique_index[0] # adding period - required to
# generally speaking, you choose frames so you have the steady state
frames = unique_index[int(0.0*nb_save):nb_save] 

if(verbose):
    print(f'Available frames :  {unique_index}')
    print(f'Max iteration : {iterations}')
    print(f'Choosen frames (for plotting) : \n{frames}')

## Proba evolution

In [14]:
fig, ax = plt.subplots(2)
# ax.set_yscale('log')
ax[0].plot(mean_proba_gas.index*dt, mean_proba_gas.values, label = 'gas')
ax[1].plot(mean_proba_walls.index*dt, mean_proba_walls.values, label = 'walls');
ax[0].legend();
ax[1].legend();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Out particles analysis

In [15]:
df_out_particles.head()

,x,y,vx,vy,vz,type
115,0.018027,0.002358,20550.091993,1319.385148,537.412261,1.0
115,0.018081,0.002164,20660.790745,571.290774,60.388648,1.0
115,0.018084,0.002321,20529.865263,1690.159952,1812.953222,1.0
116,0.018058,0.002651,20482.668143,2027.324130,386.319796,1.0
116,0.018132,0.001976,21652.645224,-361.886807,-685.016668,1.0


In [16]:
print(df_out_particles.shape[0])

9356


In [17]:
def convert_to_species(row):
    if(row['type'] == 0):
        return 'I-'
    else :
        return 'I'

In [18]:
df_out_particles['species'] = df_out_particles.apply(convert_to_species, axis = 1)

In [19]:
df_out_particles['v'] = np.sqrt(df_out_particles['vx']**2+df_out_particles['vy']**2+df_out_particles['vz']**2)
df_out_particles['energy'] = df_out_particles['v']**2
df_out_particles['angle'] = np.arctan(df_out_particles['vy']/df_out_particles['vx'])

In [20]:
ions_out = df_out_particles.loc[df_out_particles['type'] == 0]
neutral_out =  df_out_particles.loc[df_out_particles['type'] == 1]

In [21]:
results = df_out_particles[['type', 'x']].groupby('type').count();
try:
    print('Out particles neutralization percentage : {:.3} %'.format(100*results.iloc[1,0]/(results.iloc[1,0]+results.iloc[0,0])))
except:
    print('100% neutralization')
    print(results)

100% neutralization
         x
type      
1.0   9356


In [22]:
vel_results = df_out_particles[['species', 'vx', 'vy', 'vz', 'v','energy']].groupby('species').agg('mean');
vel_results

,vx,vy,vz,v,energy
species,,,,,
I,13069.610458,8.432608,-19.303842,13541.7677,2.341396e+08


In [47]:
analysis.set_fontsizes(SMALL_SIZE = 10, MEDIUM_SIZE = 12, BIGGER_SIZE = 14)

In [49]:
fig, ax = plt.subplots(constrained_layout = True)
sns.despine(fig)

sns.histplot(
    df_out_particles,
    x="vx", hue="species",
    bins = 100,
    multiple="stack",
    palette="light:m_r",
    edgecolor=".3",
    linewidth=.5,
    log_scale=False,
);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [48]:
fig, ax = plt.subplots(constrained_layout = True)
#sns.despine(fig)
ax.set_xticks([-np.pi/2, -np.pi/4, 0, np.pi/4, np.pi/2]);
ax.set_xticklabels(['-π/2', '-π/4', '0', 'π/4', 'π/2'])
sns.histplot(
    df_out_particles,
    # bins = 100,
    x="angle", hue="species",
    multiple="stack",
    palette="light:m_r",
    edgecolor=".3",
    linewidth=.5,
    log_scale=False,
)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='angle', ylabel='Count'>

### Thrust due to ions

In [50]:
print('Mean speed out : {:.3e} m/s.'.format(df_out_particles.sum()['vx']/nb_injected_particles))

Mean speed out : 1.223e+04 m/s.


In [51]:
density_ion = 3.2e17
v_mean_injection_ion = 2000 # m/s
flux_ion = 0.25*density_ion*v_mean_injection_ion

density = 3.2e19
v_mean_injection = 200 # m/s
flux = 0.25*density*v_mean_injection

# in this simulation
mean_speed_ion = df_out_particles.sum()['vx']/nb_injected_particles
mean_speed = 100 # m/s

surfasic_thrust_ion = mean_speed_ion*mass*flux_ion
surfasic_thrust = mean_speed*mass*flux
total_thrust = surfasic_thrust+surfasic_thrust_ion
print('Ratio ion / neutral : {:.3}'.format(surfasic_thrust_ion/surfasic_thrust))
print('Thrust per injection surface : {:.2e} N/m²'.format(total_thrust))

Ratio ion / neutral : 12.2
Thrust per injection surface : 4.50e-01 N/m²


In [52]:
1-1/12.2

0.9180327868852459

In [53]:
surface_pegase = 5e-2 * 4e-2 # m²
print('Pegase thrust : {:.3e} N'.format(total_thrust*surface_pegase))

Pegase thrust : 8.990e-04 N


In [54]:
V_accel = 330 # V
e = 1.6e-19
print('Isp : {:.3e} s-1'.format(mean_speed_ion/9.81))
print('Theoretical Isp : {:.3e} s-1'.format((np.sqrt(2*e*V_accel/mass))/9.81))
print('Efficiency : {:.3} %'.format(100*mean_speed_ion/np.sqrt(2*e*V_accel/mass)))

Isp : 1.246e+03 s-1
Theoretical Isp : 2.273e+03 s-1
Efficiency : 54.8 %


## df_collision_background_gas

In [55]:
df_collision_background_gas['species'] = df_collision_background_gas.apply(convert_to_species, axis = 1)

In [56]:
print('Number of collisions with background gas : {}'.format(df_collision_background_gas.shape[0]))
df_collision_background_gas.head()

Number of collisions with background gas : 5783


,x,y,type,species
1,0.000054,0.003384,0.0,I-
1,0.000029,0.004883,0.0,I-
1,0.000014,0.004905,0.0,I-
2,0.000073,0.002246,0.0,I-
3,0.000075,0.001645,0.0,I-


In [57]:
# Ion plot
sns.set_theme(style="ticks")
sns.jointplot(data = df_collision_background_gas.loc[df_collision_background_gas['type']==0], x='x', y='y', kind="hex", color="#4CB391");

# Neutral
# sns.set_theme(style="ticks")
# sns.jointplot(data = df_collision_background_gas.loc[df_collision_background_gas['type']==1], x='x', y='y', kind="hex", color="#4CB391");

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [58]:
# both
# sns.set_theme(style="ticks")
# sns.jointplot(data = df_collision_background_gas, x='x', y='y', hue="type", s = 10) # , kind="hist") #, kind="hist")

## df_collision_with_walls

In [59]:
df_collision_with_walls['species'] = df_collision_with_walls.apply(convert_to_species, axis = 1)

In [60]:
print('Number of collisions with walls : {}'.format(df_collision_with_walls.shape[0]))
df_collision_with_walls.head()

Number of collisions with walls : 74790


,x,y,type,angle,species
1,0.000024,2.998022e-07,0.0,0.738774,I-
1,0.000012,2.315137e-05,0.0,0.378493,I-
1,0.000046,4.992539e-03,0.0,1.126634,I-
1,0.000034,4.988920e-03,0.0,0.776822,I-
1,0.000045,4.997367e-03,0.0,1.155879,I-


In [61]:
ions = df_collision_with_walls.loc[df_collision_with_walls['type']==0]
fig, ax = plt.subplots()
ax.axis('equal')
scatter = ax.scatter(ions['x'], ions['y'], s = 0.005, c = ions['type']);
# produce a legend with the unique colors from the scatter
legend1 = ax.legend(*scatter.legend_elements(),
                    loc="upper right", title="Species")
ax.add_artist(legend1);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [62]:
sns.set_theme(style="ticks")
sns.jointplot(data = ions, x='x', y='y', kind="hex", color="#4CB391");

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [63]:
fig, ax = plt.subplots()
sns.despine(fig)

sns.histplot(
    df_collision_with_walls,
    # bins = 100,
    x="angle", hue="species",
    multiple="stack",
    palette="light:m_r",
    edgecolor=".3",
    linewidth=.5,
    log_scale=False,
);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [64]:
# Neutral plot
# sns.set_theme(style="ticks")
# sns.jointplot(data = df_collision_with_walls, x='x', y='y', hue="type", s = 10) # , kind="hist") #, kind="hist")

## Particles analysis
### Plotting number of particles evolution

In [65]:
df['species'] = df.apply(convert_to_species, axis = 1)

In [66]:
df.head()

,x,y,vx,vy,vz,type,species
1,0.000008,0.004777,671.406992,472.302939,745.819121,0.0,I-
1,0.000042,0.000270,2954.658408,1108.304440,1428.512151,0.0,I-
1,0.000015,0.003250,1274.714431,-52.792652,-75.409367,0.0,I-
1,0.000015,0.000041,1188.151121,548.605835,1630.475805,0.0,I-
1,0.000002,0.003980,198.520273,-294.783393,124.374629,0.0,I-


In [67]:
nb_parts = df['x'].groupby(df.index).agg('count').values

In [68]:
fig, ax = plt.subplots()
ax.plot(df.index.unique()*dt, nb_parts)
analysis.set_axis(ax, x = 'time', y = 'quantity')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [69]:
df_it = df.loc[df.index == 1000]
fig, ax = plt.subplots()
sns.despine(fig)

sns.scatterplot(
    data = df_it,
    # bins = 100,
    x="x", y ="y", hue="species", s = 5.
);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [70]:
df_it = df.loc[df.index == 1000]
fig, ax = plt.subplots()
sns.despine(fig)

sns.scatterplot(
    data = df_it,
    # bins = 100,
    x="x", y ="y", hue="vx", s = 10.
);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [71]:
dp = 1e-4
df['i'] = (df['x']/dp).astype(int)
df['j'] = (df['y']/dp).astype(int)

#### we could select only the I- - anyway

In [72]:
groups = df.loc[df['species'] == 'I-'].groupby(['i','j']).mean()

In [73]:
groups = groups.reset_index()

In [74]:
groups

,i,j,x,y,vx,vy,vz,type
0,0,0,0.000048,0.000051,1619.187341,-162.130615,-5.988500,0.0
1,0,1,0.000053,0.000150,1604.871695,61.577717,-60.107057,0.0
2,0,2,0.000051,0.000251,1632.456092,-54.418823,45.790731,0.0
3,0,3,0.000053,0.000350,1661.453291,19.082004,-46.495850,0.0
4,0,4,0.000052,0.000451,1674.159034,-22.729260,-53.469233,0.0
...,...,...,...,...,...,...,...,...
3260,169,26,0.016919,0.002632,328.780332,-1302.547893,-300.994917,0.0
3261,169,27,0.016908,0.002734,926.315291,-3349.414753,378.634895,0.0
3262,170,23,0.017005,0.002378,1034.874653,2573.527778,-1320.224328,0.0
3263,170,24,0.017024,0.002473,-124.943925,633.636917,-686.489167,0.0


In [75]:
groups['i'].max()

170

In [76]:
fig, ax = plt.subplots()

values = {
    'width' : 3*dp,
    'headwidth' : 3,
    'headlength' : 5
}

ax.quiver(dp*groups['i'], dp*groups['j'], groups['vx'], groups['vy'], groups['vx'], **values);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [77]:
U,V = np.zeros((groups['j'].max()-groups['j'].min(),groups['i'].max()-groups['i'].min()), dtype = float), np.zeros((groups['j'].max()-groups['j'].min(),groups['i'].max()-groups['i'].min()), dtype = float)
X, Y = np.arange(groups['i'].min(), groups['i'].max()),  np.arange(groups['j'].min(), groups['j'].max())

for x in X:
    for y in Y:
        row = groups.loc[(groups['i'] == x) &  (groups['j'] == y)]
        try :
            U[y,x] = row['vx']
            V[y,x] = row['vy']
        except ValueError as e:
            pass

In [78]:
print(U.shape, V.shape)
print(X.shape, Y.shape)
speed = np.sqrt(U**2+V**2)
lw = 2 * speed / (speed.max()+1e-8)

(49, 170) (49, 170)
(170,) (49,)


In [79]:
fig, ax = plt.subplots(constrained_layout=True)
# ax.axis('equal')
h = ax.streamplot(x=dp*X,y=dp*Y,u=U,v=V, density = 3, color=U, linewidth=lw, cmap='plasma'); 
cbar = fig.colorbar(h.lines, ax=ax)
analysis.set_axis(ax, x = 'x', y = 'y');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …